# Ceneo Scraper

## Components Of Single Opinion


|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|["span.user-post__author-recomendation > em|author|
|author’s recommendation|["span.user-post__score-count"]|recomendation|
|score expressed in number of stars|["div.user-post__score-count"]|score|
|opinion’s content|["div.user-post__text"]|content|
|list of product advantages|["div.review.feture__title--positives ~ div.review-feture__item"]|pros|
|list of product disadvantages|["div.review.feture__title--negatives ~ div.review-feture__item"]|cons|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|unhelpful|
|publishing date|span.user-post__published > time:nth-child(1)|publish_date|
|purchase date|span.user-post__published > time:nth-child(2)|purchase_date|


## Structure of single opinion

In [28]:
def rate(score):
    rating = score.split("/")
    return float(score[0].replace(",","."))/float(score[1])
def recommend(recommendation):
    return True if recommendation == "Polecam" else False if recommendation == "Nie polecam" else None

In [2]:
selectors = {
    "opinion_id" : [None, "data-entry-id"],
    "author" : ["span.user-post__author-recomendation"],
    "recomendation" : ["span.user-post__score-count"],
    "score" : ["div.user-post__score-count"],
    "content" : ["div.user-post__text"],
    "pros" : ["div.review.feture__title--positives ~ div.review-feture__item", None, True],
    "cons" : ["div.revuew.fetere__title--negatives ~ div.review-feture__item", None, True],
    "helpful" : ["button.vote-yes > span"],
    "unhelpful" : ["button.vote-no > span"],
    "publish_date" : ["span.user-post__published > time:nth-child(1)", "datetime"],
    "purchase_date" : ["span.user-post__published > time:nth-child(2)","datetime"],
}

# Loading libraries

In [31]:
import json
import os
import requests
from bs4 import BeautifulSoup

## Transformations

In [ ]:
transformations = {
        "score" : rate,
        "recomendation" : recommend,
        "helpful" : int,
        "unhelpful" : int,
}

## Function to extract data from html code

In [32]:
def extract(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute] for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().stip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.get_text().strip()

## URL adress for first page of a product


In [33]:
product_id = "41586775"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

## Extracting opinions from HTML code

In [30]:

all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        for key, value in transformations.items():
                single_opinion[key] = map(value, single_opinion[key])
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl/" + extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = None

## Saving all opinions to JSON file

In [34]:
if os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_id}.json", "w", encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()

FileNotFoundError: [Errno 2] No such file or directory: 'opinions/41586775.json'

In [9]:
for index, single_opinion in enumerate(all_opinions):
    for key, value in transformations.items():

'https://www.ceneo.pl/41586775#tab=reviews'